### Cab-Driver Agent

In [1]:
# Importing libraries
import numpy as np
import random
import math
from collections import deque
import collections
import pickle
import os
from itertools import permutations,product
# for building DQN model
from keras import layers
from keras import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam
import keras.engine.saving as save_model

# for plotting graphs
import matplotlib.pyplot as plt
import pylab

# Import the environment
from Env import CabDriver


Using TensorFlow backend.


#### Defining Time Matrix

In [2]:
# Loading the time matrix provided
Time_matrix = np.load("TM.npy")

#### Tracking the state-action pairs for checking convergence


In [3]:
#Defining a function to save the Q-dictionary as a pickle file
def save_pickle(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [4]:
#define epsilon_decay strategy
def epsilon_decay(total_steps, step):
    epsilon = abs(np.exp((-np.pi/total_steps)*step))
    return epsilon

### Agent Class

If you are using this framework, you need to fill the following to complete the following code block:
1. State and Action Size
2. Hyperparameters
3. Create a neural-network model in function 'build_model()'
4. Define epsilon-greedy strategy in function 'get_action()'
5. Complete the function 'append_sample()'. This function appends the recent experience tuple <state, action, reward, new-state> to the memory
6. Complete the 'train_model()' function with following logic:
   - If the memory size is greater than mini-batch size, you randomly sample experiences from memory as per the mini-batch size and do the following:
      - Initialise your input and output batch for training the model
      - Calculate the target Q value for each sample: reward + gamma*max(Q(s'a,))
      - Get Q(s', a) values from the last trained model
      - Update the input batch as your encoded state and output batch as your Q-values
      - Then fit your DQN model using the updated input and output batch.

In [ ]:
#this cell contains the agent class

class DQNAgent:
    def __init__(self, state_size, action_size, action_map, discount_factor=0.95, learning_rate=0.01,
                       epsilon=0.99, epsilon_decay=0.99, epsilon_min=0.01):
        # Define size of state and action
        self.state_size = state_size
        self.action_size = action_size

        # Write here: Specify you hyper parameters for the DQN
        self.discount_factor = discount_factor
        self.learning_rate = learning_rate        
        self.epsilon_max = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        self.model_history = None
        self.action_map = action_map
        self.batch_size = 32
        #self.batch_size = 1
        # create replay memory using deque
        self.memory = deque(maxlen=4096)

        # create main model and target model
        self.model = self.build_model()
    

    # approximate Q function using Neural Network
    def build_model(self):
        model = Sequential()
        # Write your code here: Add layers to your neural nets       

        # hidden layers
        model.add(Dense(32, input_dim=self.state_size, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dense(32, activation='relu', kernel_initializer='he_uniform'))
        #model.add(Dense(32, activation='relu', kernel_initializer='he_uniform'))

        # the output layer: output is of size num_actions
        model.add(Dense(self.action_size, activation='relu', kernel_initializer='he_uniform'))     
        model.compile(loss='mse',optimizer=Adam(lr=self.learning_rate))
        model.summary()
        return model


    def get_action(self, cstate, all_actions, pos_act_ind):
    # Write your code here:
    # get action from model using epsilon-greedy policy
    # Decay in ε after we generate each sample from the environment
        actions = all_actions[pos_act_ind]
        if np.random.rand() <= self.epsilon_max:
            # explore: choose a random action from all possible actions
            mode = 'Exploring'
            action = random.choice(actions)
        else:
            # choose the action with the highest q(s, a)
            # the first index corresponds to the batch size, so
            # reshape state to (1, state_size) so that the first index corresponds to the batch size
            mode = 'Exploiting'
            #cstate = cstate.reshape(1, self.state_size) 
            q_value = self.model.predict(x=cstate)
            Q_val_for_actions = q_value[0][pos_act_ind]
            max_index = np.argmax(Q_val_for_actions)
            print(q_value.shape, Q_val_for_actions, 'Index ', max_index, np.max(Q_val_for_actions))
            action = actions[max_index]
        print(mode, 'Available actions ', actions, ' Selected action ', action)    
        return action
        

    def append_sample(self, state, action, reward, next_state, done):
    # Write your code here:
    # save sample <s,a,r,s'> to the replay memory
        self.memory.append((state, action, reward, next_state, done))
    
    
    # pick samples randomly from replay memory (with batch_size) and train the network
    def train_model(self, states_to_be_tracked):
        if len(self.memory) > self.batch_size:
            # Sample batch from the memory
            mini_batch = random.sample(self.memory, self.batch_size)
            update_input = np.zeros((self.batch_size, self.state_size))
            update_output = np.zeros((self.batch_size, self.state_size))
            
            actions, rewards, done = [], [], []
            
            for i in range(self.batch_size):
                state, action, reward, next_state, done_boolean = mini_batch[i]
                # Write your code from here
                # 1. Identify the next action 
                update_input[i] = env.state_encod_arch1(state)
                actions.append(action)
                rewards.append(reward)
                update_output[i] = env.state_encod_arch1(next_state)
                done.append(done_boolean)
                
            # 2. Get the target for the Q-network
            
            target = self.model.predict(update_input)
            target_qval = self.model.predict(update_output)
            
            #3. Update our target rewards
            for i in range(self.batch_size):
                index = self.action_map[tuple(actions[i])]
                if done[i]:
                    target[i][index] = rewards[i]
                else: # non-terminal state
                    max_qvalue = np.max(target_qval[i])
                    target[i][index] = rewards[i] + self.discount_factor * max_qvalue
                    if states_to_be_tracked and (mini_batch[i][0], tuple(actions[i])) in states_to_be_tracked:
                        states_to_be_tracked[(mini_batch[i][0], tuple(actions[i]))] = target_qval
                        
                
        # 4. Fit your model and track the loss values
            return self.model.fit(update_input, target, batch_size=self.batch_size, epochs=1, verbose=0)
            
            
    def save(self, name):
        self.model.save(name)

In [ ]:
# to store rewards in each episode
rewards_per_episode, episodes  = [], []

# make dir to store model weights
if not os.path.exists("saved_model_weights"):
    os.mkdir("saved_model_weights")

# n_episodes
n_episodes = 1000

### DQN block

In [ ]:
 # Call all the initialised variables of the environment
env = CabDriver()
#Call the DQN agent
dqn = DQNAgent(env.state_size, env.action_size, env.action_map)
states_to_be_tracked_keys = product(env.state_space[:210], env.action_list[:20])
states_to_be_tracked = {k:[] for k in states_to_be_tracked_keys}

for episode in range(n_episodes):

    # Write code here
    # Call the environment
   
    _,_,curr_state = env.reset()
    state_size = env.state_size
    step = 0
    
   
    #action_size = len(actions)
    reward = 0
    curr_time = 0
    
    
    terminal_state = False
    print("Episode :", episode)
    
    while not terminal_state:
        # Write your code here
        # 1. Pick epsilon-greedy action from possible actions for the current state
        step +=1
        encoded_state = env.state_encod_arch1(curr_state)
        pos_act_ind, actions = env.requests(curr_state)
        action = dqn.get_action(encoded_state, env.action_space, pos_act_ind)
        
        # 2. Evaluate your reward and next state
        reward = reward + env.reward_func(curr_state, action, Time_matrix)
        next_state = env.next_state_func(curr_state,action,Time_matrix)
        
        pickup_loc = action[0]
        drop_loc = action[1]
        current_loc = curr_state[0]
        time = curr_state[1]
        day = curr_state[2]
        #calculate time increase only on different pickup and drop points
        if pickup_loc != drop_loc:
            curr_time = curr_time + Time_matrix[current_loc][pickup_loc][time][day]
            time = next_state[1]
            day = next_state[2]
            curr_time = curr_time + Time_matrix[pickup_loc][drop_loc][time][day]
        else:
            curr_time += 1.0
        
        # 3. Append the experience to the memory
        dqn.append_sample(curr_state, action, reward, next_state, terminal_state)
        curr_state = next_state
        
        # 4. Train the model by calling function agent.train_model
        history = dqn.train_model(states_to_be_tracked)
        # 5. Keep a track of rewards, Q-values, loss
        print("episode:", episode, "  score:", reward, "  memory length:",
                      len(dqn.memory), "  epsilon:", dqn.epsilon_max)
        
        if curr_time >= 24*30:
            terminal_state = True
    
    # store total reward obtained in this episode
    rewards_per_episode.append(reward)
    episodes.append(episode)
    #pylab.plot(episode, rewards_per_episode, 'b')
    #pylab.savefig("./cab_dqn.png")
        
    
    dqn.epsilon_max =  epsilon_decay(n_episodes, episode)
    #save model for every 50 episodes
    if episode % 50 == 0:
            dqn.model.save_weights("./saved_model_weights/driver_dqn.h5")
            save_pickle(save_model.pickle_model(dqn.model), f"saved_pickle_files/driver_dqn_models_{episode}")
            
        
        

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                26912     
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_3 (Dense)              (None, 21)                693       
Total params: 28,661
Trainable params: 28,661
Non-trainable params: 0
_________________________________________________________________
Episode : 0
Exploring Available actions  [[3 2]
 [1 0]
 [1 2]
 [1 3]
 [0 1]
 [1 4]
 [0 0]]  Selected action  [1 2]
episode: 0   score: -31.0   memory length: 1   epsilon: 0.99
Exploring Available actions  [[1 0]
 [0 0]]  Selected action  [0 0]
episode: 0   score: -36.0   memory length: 2   epsilon: 0.99
Exploring Available actions  [[0 4]
 [1 0]
 [0 0]]  Selected action  [0 0]
episode: 0   score: -4

episode: 0   score: -99.0   memory length: 62   epsilon: 0.99
Exploring Available actions  [[0 3]
 [3 4]
 [3 0]
 [0 2]
 [4 1]
 [0 1]
 [2 0]
 [3 1]
 [0 4]
 [3 2]
 [1 3]
 [0 0]]  Selected action  [3 4]
episode: 0   score: -80.0   memory length: 63   epsilon: 0.99
Exploring Available actions  [[4 1]
 [2 4]
 [0 4]
 [0 0]]  Selected action  [0 0]
episode: 0   score: -85.0   memory length: 64   epsilon: 0.99
Exploring Available actions  [[2 4]
 [3 4]
 [1 0]
 [4 1]
 [3 1]
 [3 0]
 [2 3]
 [1 2]
 [0 0]]  Selected action  [3 4]
episode: 0   score: -97.0   memory length: 65   epsilon: 0.99
Exploring Available actions  [[0 2]
 [1 3]
 [4 1]
 [1 0]
 [3 2]
 [2 1]
 [4 3]
 [1 4]
 [3 4]
 [4 0]
 [0 0]]  Selected action  [0 0]
episode: 0   score: -102.0   memory length: 66   epsilon: 0.99
Exploring Available actions  [[1 2]
 [2 0]
 [3 4]
 [3 1]
 [3 2]
 [0 4]
 [2 4]
 [0 0]]  Selected action  [0 4]
episode: 0   score: -102.0   memory length: 67   epsilon: 0.99
Exploring Available actions  [[3 2]
 [4 1]
 [0 4

episode: 0   score: -150.0   memory length: 121   epsilon: 0.99
Exploring Available actions  [[3 4]
 [2 0]
 [3 1]
 [2 3]
 [3 2]
 [1 3]
 [2 4]
 [4 1]
 [0 4]
 [1 4]
 [1 0]
 [0 1]
 [1 2]
 [0 3]
 [4 2]
 [0 0]]  Selected action  [0 4]
episode: 0   score: -181.0   memory length: 122   epsilon: 0.99
Exploring Available actions  [[0 2]
 [4 0]
 [0 3]
 [3 2]
 [2 1]
 [4 1]
 [0 1]
 [2 0]
 [0 0]]  Selected action  [0 3]
episode: 0   score: -182.0   memory length: 123   epsilon: 0.99
Exploring Available actions  [[0 1]
 [2 3]
 [4 0]
 [3 0]
 [0 4]
 [4 3]
 [3 2]
 [1 0]
 [1 2]
 [0 0]]  Selected action  [0 4]
episode: 0   score: -171.0   memory length: 124   epsilon: 0.99
Exploring Available actions  [[0 2]
 [0 1]
 [1 4]
 [3 1]
 [0 0]]  Selected action  [1 4]
episode: 0   score: -171.0   memory length: 125   epsilon: 0.99
Exploring Available actions  [[2 4]
 [3 4]
 [4 1]
 [2 1]
 [4 3]
 [0 1]
 [1 4]
 [1 3]
 [1 0]
 [0 0]]  Selected action  [4 3]
episode: 0   score: -163.0   memory length: 126   epsilon: 0

episode: 1   score: 26.0   memory length: 185   epsilon: 1.0
Exploring Available actions  [[4 2]
 [1 4]
 [3 2]
 [4 3]
 [3 1]
 [1 0]
 [2 4]
 [0 3]
 [0 0]]  Selected action  [1 0]
episode: 1   score: 26.0   memory length: 186   epsilon: 1.0
Exploring Available actions  [[2 4]
 [0 0]]  Selected action  [2 4]
episode: 1   score: -9.0   memory length: 187   epsilon: 1.0
Exploring Available actions  [[3 4]
 [2 3]
 [4 0]
 [0 2]
 [1 2]
 [4 3]
 [3 0]
 [0 1]
 [0 0]]  Selected action  [0 2]
episode: 1   score: 7.0   memory length: 188   epsilon: 1.0
Exploring Available actions  [[4 1]
 [3 0]
 [0 4]
 [0 0]]  Selected action  [0 4]
episode: 1   score: -9.0   memory length: 189   epsilon: 1.0
Exploring Available actions  [[4 0]
 [1 0]
 [2 3]
 [4 3]
 [2 1]
 [2 4]
 [0 3]
 [0 0]]  Selected action  [4 0]
episode: 1   score: -5.0   memory length: 190   epsilon: 1.0
Exploring Available actions  [[0 1]
 [3 0]
 [0 2]
 [0 0]]  Selected action  [3 0]
episode: 1   score: 4.0   memory length: 191   epsilon: 1.0

episode: 1   score: 20.0   memory length: 239   epsilon: 1.0
Exploring Available actions  [[2 4]
 [1 3]
 [0 2]
 [2 0]
 [3 0]
 [0 3]
 [1 4]
 [4 1]
 [4 2]
 [1 0]
 [0 4]
 [0 0]]  Selected action  [2 0]
episode: 1   score: 19.0   memory length: 240   epsilon: 1.0
Exploring Available actions  [[3 1]
 [3 4]
 [0 0]]  Selected action  [3 1]
episode: 1   score: 12.0   memory length: 241   epsilon: 1.0
Exploring Available actions  [[3 2]
 [1 3]
 [0 3]
 [4 3]
 [4 2]
 [1 0]
 [4 0]
 [1 4]
 [2 0]
 [2 1]
 [3 0]
 [0 1]
 [0 2]
 [0 0]]  Selected action  [0 2]
episode: 1   score: -15.0   memory length: 242   epsilon: 1.0
Exploring Available actions  [[0 1]
 [0 0]]  Selected action  [0 0]
episode: 1   score: -20.0   memory length: 243   epsilon: 1.0
Exploring Available actions  [[3 1]
 [0 2]
 [2 0]
 [4 1]
 [3 2]
 [0 0]]  Selected action  [3 2]
episode: 1   score: -28.0   memory length: 244   epsilon: 1.0
Exploring Available actions  [[2 0]
 [3 2]
 [4 0]
 [0 0]]  Selected action  [4 0]
episode: 1   score: 

...

episode: 91   score: -354.0   memory length: 4096   epsilon: 0.7537132119564671
(1, 21) [0. 0. 0. 0. 0. 0. 0.] Index  0 0.0
Exploiting Available actions  [[4 0]
 [1 3]
 [2 0]
 [3 1]
 [3 4]
 [1 4]
 [0 0]]  Selected action  [4 0]
episode: 91   score: -359.0   memory length: 4096   epsilon: 0.7537132119564671
(1, 21) [0. 0. 0.] Index  0 0.0
Exploiting Available actions  [[2 0]
 [0 3]
 [0 0]]  Selected action  [2 0]
episode: 91   score: -361.0   memory length: 4096   epsilon: 0.7537132119564671
Exploring Available actions  [[4 3]
 [0 0]]  Selected action  [4 3]
episode: 91   score: -379.0   memory length: 4096   epsilon: 0.7537132119564671
Exploring Available actions  [[2 0]
 [1 0]
 [1 2]
 [0 0]]  Selected action  [1 0]
episode: 91   score: -372.0   memory length: 4096   epsilon: 0.7537132119564671
Exploring Available actions  [[3 0]
 [1 3]
 [0 4]
 [0 0]]  Selected action  [3 0]
episode: 91   score: -383.0   memory length: 4096   epsilon: 0.7537132119564671
Exploring Available actions  [[2

episode: 92   score: -22.0   memory length: 4096   epsilon: 0.7513490676005719
Exploring Available actions  [[0 3]
 [0 2]
 [4 3]
 [4 0]
 [1 3]
 [1 4]
 [2 0]
 [4 1]
 [4 2]
 [0 0]]  Selected action  [4 3]
episode: 92   score: -2.0   memory length: 4096   epsilon: 0.7513490676005719
Exploring Available actions  [[2 0]
 [0 4]
 [4 0]
 [2 3]
 [0 0]]  Selected action  [2 0]
episode: 92   score: -15.0   memory length: 4096   epsilon: 0.7513490676005719
Exploring Available actions  [[1 0]
 [0 0]]  Selected action  [1 0]
episode: 92   score: -38.0   memory length: 4096   epsilon: 0.7513490676005719
Exploring Available actions  [[0 2]
 [0 1]
 [0 3]
 [0 0]]  Selected action  [0 1]
episode: 92   score: -26.0   memory length: 4096   epsilon: 0.7513490676005719
Exploring Available actions  [[2 4]
 [3 4]
 [0 2]
 [2 0]
 [0 3]
 [4 3]
 [1 2]
 [3 2]
 [3 0]
 [4 2]
 [0 4]
 [0 0]]  Selected action  [3 0]
episode: 92   score: -19.0   memory length: 4096   epsilon: 0.7513490676005719
Exploring Available action

episode: 92   score: -189.0   memory length: 4096   epsilon: 0.7513490676005719
(1, 21) [0. 0. 0.] Index  0 0.0
Exploiting Available actions  [[2 1]
 [0 4]
 [0 0]]  Selected action  [2 1]
episode: 92   score: -149.0   memory length: 4096   epsilon: 0.7513490676005719
Exploring Available actions  [[3 0]
 [1 3]
 [4 3]
 [3 2]
 [2 0]
 [3 1]
 [3 4]
 [1 4]
 [0 1]
 [0 0]]  Selected action  [3 4]
episode: 92   score: -153.0   memory length: 4096   epsilon: 0.7513490676005719
Exploring Available actions  [[2 3]
 [0 1]
 [1 2]
 [3 2]
 [1 4]
 [3 1]
 [0 4]
 [2 1]
 [2 4]
 [0 0]]  Selected action  [0 0]
episode: 92   score: -158.0   memory length: 4096   epsilon: 0.7513490676005719
Exploring Available actions  [[0 2]
 [4 3]
 [0 4]
 [2 0]
 [1 4]
 [2 1]
 [4 2]
 [3 1]
 [3 2]
 [0 0]]  Selected action  [2 1]
episode: 92   score: -150.0   memory length: 4096   epsilon: 0.7513490676005719
(1, 21) [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] Index  0 0.0
Exploiting Available actions  [[1 3]
 [2 0]
 [4 1]
 [3 4]
 [0 2]

episode: 93   score: -6.0   memory length: 4096   epsilon: 0.74899233876884
Exploring Available actions  [[0 4]
 [1 0]
 [1 3]
 [3 1]
 [2 1]
 [0 0]]  Selected action  [1 0]
episode: 93   score: -18.0   memory length: 4096   epsilon: 0.74899233876884
(1, 21) [0. 0. 0.] Index  0 0.0
Exploiting Available actions  [[0 2]
 [0 1]
 [0 0]]  Selected action  [0 2]
episode: 93   score: -6.0   memory length: 4096   epsilon: 0.74899233876884
(1, 21) [0. 0. 0. 0. 0. 0. 0. 0. 0.] Index  0 0.0
Exploiting Available actions  [[2 3]
 [1 3]
 [3 2]
 [2 1]
 [0 3]
 [0 2]
 [4 3]
 [0 4]
 [0 0]]  Selected action  [2 3]
episode: 93   score: 26.0   memory length: 4096   epsilon: 0.74899233876884
(1, 21) [0. 0. 0. 0. 0. 0.] Index  0 0.0
Exploiting Available actions  [[0 1]
 [0 3]
 [1 2]
 [0 4]
 [3 4]
 [0 0]]  Selected action  [0 1]
episode: 93   score: 12.0   memory length: 4096   epsilon: 0.74899233876884
Exploring Available actions  [[0 2]
 [2 0]
 [1 2]
 [3 1]
 [1 0]
 [1 4]
 [0 3]
 [3 4]
 [1 3]
 [2 4]
 [0 0]]  S

episode: 93   score: 120.0   memory length: 4096   epsilon: 0.74899233876884
Exploring Available actions  [[0 1]
 [2 4]
 [3 0]
 [1 4]
 [3 4]
 [2 3]
 [1 3]
 [1 2]
 [4 1]
 [0 2]
 [3 1]
 [0 0]]  Selected action  [3 0]
episode: 93   score: 109.0   memory length: 4096   epsilon: 0.74899233876884
(1, 21) [0. 0. 0. 0.] Index  0 0.0
Exploiting Available actions  [[1 4]
 [3 2]
 [3 4]
 [0 0]]  Selected action  [1 4]
episode: 93   score: 78.0   memory length: 4096   epsilon: 0.74899233876884
Exploring Available actions  [[0 3]
 [2 0]
 [4 0]
 [0 1]
 [1 2]
 [4 2]
 [1 3]
 [0 0]]  Selected action  [4 2]
episode: 93   score: 110.0   memory length: 4096   epsilon: 0.74899233876884
(1, 21) [0. 0.] Index  0 0.0
Exploiting Available actions  [[3 0]
 [0 0]]  Selected action  [3 0]
episode: 93   score: 82.0   memory length: 4096   epsilon: 0.74899233876884
Exploring Available actions  [[3 0]
 [0 0]]  Selected action  [0 0]
episode: 93   score: 77.0   memory length: 4096   epsilon: 0.74899233876884
(1, 21) [

In [ ]:
# make directory
if not os.path.exists("saved_pickle_files"):
    os.mkdir("saved_pickle_files")

# save rewards_per_episode
save_pickle(rewards_per_episode, "saved_pickle_files/rewards_per_episode")


# plot results
with open('saved_pickle_files/rewards_per_episode.pkl', 'rb') as f:
    rewards_per_episode = pickle.load(f)

plt.plot(list(range(len(rewards_per_episode))), rewards_per_episode)
plt.xlabel("episode number")
plt.ylabel("reward per episode")

# save plots in saved_plots/ directory
plt.savefig('rewards.png')

print("Average reward of last 100 episodes is {0}".format(np.mean(rewards_per_episode[-100:])))

## Tracking convergence

In [ ]:
j = 0
tracked_state_action = list(states_to_be_tracked.keys())
for i in range(1, 10):
    plt.figure(j, figsize=(16, 7))
    state_action = tracked_state_action[i]
    if not len(states_to_be_tracked[state_action]):
        continue
    xaxis = np.asarray(range(0, len(states_to_be_tracked[state_action])))
    plt.title('state={0} action={1}'.format(state_action[0], state_action[1]))
    plt.plot(xaxis,np.asarray(states_to_be_tracked[state_action]))
    j = (j + 1) % 9
    plt.show()

#### Epsilon-decay sample function

<div class="alert alert-block alert-info">
Try building a similar epsilon-decay function for your model.
</div>

In [ ]:
total_steps = 50
time = np.arange(0, total_steps)
epsilon = []
x = 0.99
for step in range(0,total_steps):
    x = abs(np.exp((-np.pi/total_steps)*step))
    epsilon.append(x)

In [ ]:
plt.plot(time, epsilon)
plt.show()